In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from joblib import Parallel, delayed
import glob
import time
from tqdm import tqdm
import os
import json
from lvm_prediction import predict
import pingouin as pg

## Filter bad data 

In [2]:
import os
import glob
import json
import numpy as np

files = glob.glob('/home/ks2823/Microbiomap/in_silico_replicates_v7/*.json')

for file in files:
    try:
        num_niches = int(file.split('_')[-1].split('.')[0])
    except (ValueError, IndexError):
        # Skips files that don't match the expected naming convention
        continue

    # if num_niches < 5:
    #     continue

    print(f"Processing file: {file} with {num_niches} niches")

    data_list = []
    if os.path.exists(file):
        with open(file, 'r') as f:
            for line in f:
                try:
                    item = json.loads(line)
                    data_list.append(item)
                except json.JSONDecodeError:
                    # Skips lines that are not valid JSON
                    continue

    filtered_data_list = []
    for dataset in data_list:
        if 'inflow' in dataset:
            try:
                inflow = np.array(dataset['inflow'])
                if inflow.shape[1] == 250:
                    filtered_data_list.append(dataset)
            except IndexError:
                # Skips entries where 'inflow' is not a 2D array
                continue

    if len(data_list) != len(filtered_data_list):
        print(f"  Removed {len(data_list) - len(filtered_data_list)} entries from {os.path.basename(file)}")

        # with open(file, 'w') as f:
        #     for item in filtered_data_list:
        #         f.write(json.dumps(item) + '\n')
    else:
        print(f"  No entries were removed from {os.path.basename(file)}")

Processing file: /home/ks2823/Microbiomap/in_silico_replicates_v7/in_silico_replicates_1.json with 1 niches
  No entries were removed from in_silico_replicates_1.json
Processing file: /home/ks2823/Microbiomap/in_silico_replicates_v7/in_silico_replicates_2.json with 2 niches
  No entries were removed from in_silico_replicates_2.json
Processing file: /home/ks2823/Microbiomap/in_silico_replicates_v7/in_silico_replicates_3.json with 3 niches
  No entries were removed from in_silico_replicates_3.json
Processing file: /home/ks2823/Microbiomap/in_silico_replicates_v7/in_silico_replicates_4.json with 4 niches
  No entries were removed from in_silico_replicates_4.json
Processing file: /home/ks2823/Microbiomap/in_silico_replicates_v7/in_silico_replicates_5.json with 5 niches
  No entries were removed from in_silico_replicates_5.json


## Check sims

In [3]:
files = glob.glob('/home/ks2823/Microbiomap/in_silico_replicates_v7/*.json')
num_processed = 0
num_under = 0
for file in files:
    num_niches = int(file.split('_')[-1].split('.')[0])
    # if num_niches > 5:
    #     continue
    print("Num niches:", num_niches)

    data_list = []
    if os.path.exists(file):
        with open(file, 'r') as f:
            for line in f:
                item = json.loads(line)
                data_list.append(item)
    
    best_dist_list = []    
    for dataset in data_list:
        # print(dataset.keys())
        project_id = dataset['label']

        best_dist = dataset['best_dist']
        best_dist_list.append(best_dist)
    best_dist_list = np.array(best_dist_list)
    num_processed += best_dist_list.size
    num_under += np.sum(best_dist_list<0.3)
    print(f'Max: {np.max(best_dist_list):.2f}, Mean: {np.mean(best_dist_list):.2f}, Min: {np.min(best_dist_list):.2f}, Percent under 0.3: {(np.sum(best_dist_list<0.3)/best_dist_list.size):.2f}, No. of projects: {best_dist_list.size}\n')
print(f'\nPercentage of projects processed: {num_processed/5}')
print(f'\nPercentage of projects under 0.3: {100*num_under/num_processed:.1f}')

Num niches: 1
Max: 0.97, Mean: 0.28, Min: 0.04, Percent under 0.3: 0.66, No. of projects: 50

Num niches: 10
Max: 0.94, Mean: 0.30, Min: 0.06, Percent under 0.3: 0.72, No. of projects: 50

Num niches: 2
Max: 1.04, Mean: 0.38, Min: 0.05, Percent under 0.3: 0.42, No. of projects: 50

Num niches: 3
Max: 0.94, Mean: 0.33, Min: 0.11, Percent under 0.3: 0.62, No. of projects: 50

Num niches: 4
Max: 1.13, Mean: 0.33, Min: 0.10, Percent under 0.3: 0.60, No. of projects: 50

Num niches: 5
Max: 0.90, Mean: 0.33, Min: 0.11, Percent under 0.3: 0.52, No. of projects: 50

Num niches: 6
Max: 1.18, Mean: 0.33, Min: 0.03, Percent under 0.3: 0.64, No. of projects: 50

Num niches: 7
Max: 1.27, Mean: 0.31, Min: 0.06, Percent under 0.3: 0.66, No. of projects: 50

Num niches: 8
Max: 1.04, Mean: 0.28, Min: 0.04, Percent under 0.3: 0.68, No. of projects: 50

Num niches: 9
Max: 1.34, Mean: 0.36, Min: 0.06, Percent under 0.3: 0.54, No. of projects: 50


Percentage of projects processed: 100.0

Percentage of pro

Max: 0.97, Mean: 0.61, Min: 0.26, Percent under 0.45: 0.50, No. of projects: 4

## Initial calculations

In [2]:
def calc_taylor_slope(data):
    # Placeholder for actual slope calculation logic
    # Assuming data to be a 2D array-like structure
    means = data.mean(axis=0)
    variances = data.var(axis=0)
    slope = np.polyfit(np.log(means), np.log(variances), 1)[0]
    return slope

def calc_alpha(data):
    return sp.stats.entropy(data, axis=1).mean()

def fit_exp(data):
    data = data[:5]
    data = data/data[0]
    model = LinearRegression()
    try:
        model.fit(np.arange(len(data)).reshape(-1, 1), np.log(data))
    except:
        return np.nan
    exponent = -model.coef_[0]
    return exponent

def participation_ratio(eigenvalues):
    return (np.sum(eigenvalues) ** 2) / np.sum(eigenvalues ** 2)

def get_shuffled(data, normalization=True):
    permuted_data = np.array([np.random.permutation(x) for x in data.T]).T
    if normalization:
        permuted_data /= np.sum(permuted_data, axis=1)[:, np.newaxis]
    return permuted_data

def calc_env_dim(met):
    met[met<=0] = 0.1*np.min(met[met>0])
    log_met = np.log(met) 
    
    met = met - np.mean(met, axis=0)[np.newaxis, :]
    met = met[:, ~(np.std(met, axis=0)==0)]
    met_std = np.std(met, axis=0)
    assert np.all(met_std!=0), "Met is not filtered properly"
       
    log_met = log_met - np.mean(log_met, axis=0)[np.newaxis, :]
    log_met = log_met[:, ~(np.std(log_met, axis=0)==0)]
    log_met_std = np.std(log_met, axis=0)
    assert np.all(log_met_std!=0), "Log-Met is not filtered properly"
    
    met = met/met_std[np.newaxis, :]
    log_met = log_met/log_met_std[np.newaxis, :]
    
    met_shuffled = get_shuffled(met, normalization=False)
    log_met_shuffled = get_shuffled(log_met, normalization=False)
    
    svd = np.linalg.svd(met, compute_uv=False, full_matrices=False)
    svd_log = np.linalg.svd(log_met, compute_uv=False, full_matrices=False)
    svd_shuffled = np.linalg.svd(met_shuffled, compute_uv=False, full_matrices=False)
    svd_shuffled_log = np.linalg.svd(log_met_shuffled, compute_uv=False, full_matrices=False)
    
    
    env_dim = 1/fit_exp(svd)
    env_dim_log = 1/fit_exp(svd_log)
    env_pr = participation_ratio(svd)
    env_pr_log = participation_ratio(svd_log)
    env_shuffled_pr = participation_ratio(svd_shuffled)
    env_shuffled_pr_log = participation_ratio(svd_shuffled_log)
    env_pr_corrected = env_pr - env_shuffled_pr
    env_pr_log_corrected = env_pr_log - env_shuffled_pr_log
    
    return env_dim, env_dim_log, env_pr, env_pr_log, env_pr_corrected, env_pr_log_corrected

def calc_generalists(crm, mean_abu):
    # sparsity = np.sum(crm)/crm.size    
    # mean_num_resources = crm.shape[1]*sparsity
    # weighted_num_resources = np.dot(crm.sum(axis=1), mean_abu)
    # weighted_to_mean_ratio = weighted_num_resources/mean_num_resources
    
    # return weighted_to_mean_ratio
    return np.dot(mean_abu, crm.sum(axis=1))/np.mean(crm.sum(axis=1))  

In [3]:
in_silico_checkpoint_file = '/home/ks2823/Microbiomap/checkpoints/in_silico_v7.csv'
files = glob.glob('/home/ks2823/Microbiomap/in_silico_replicates_v7/*.json')

in_silico_df = pd.DataFrame() 

# try:
#     in_silico_df = pd.read_csv(in_silico_checkpoint_file, index_col=0)
# except FileNotFoundError:
#     print(f"Checkpoint file not found: {in_silico_checkpoint_file}. Starting with an empty DataFrame.")
# except Exception as e:
#     print(f"An error occurred while reading the checkpoint file: {e}")

for file in files:
    
    num_niches = int(file.split('_')[-1].split('.')[0])
    print("Num niches:", num_niches)

    processed_projects = []
    if not in_silico_df.empty:
        processed_projects = list(in_silico_df[in_silico_df['num_niches'] == num_niches]['label'])
        print(f'Already processed {len(processed_projects)}')

    data_list = []
    if os.path.exists(file):
        with open(file, 'r') as f:
            for line in f:
                item = json.loads(line)
                data_list.append(item)

    for dataset in tqdm(data_list):
        # print(dataset.keys())
        if dataset['best_dist'] > 0.3:
            continue
        project_id = dataset['label']
        
        if project_id in processed_projects:
            # print(f'Skipping {project_id}')
            continue

        abu = np.array(dataset['data'])
        metadata = np.array(dataset['metadata'])
        
        inflow = np.array(dataset['inflow'])
        total_abu = np.sum(abu, axis=1)
        rel_abu = abu/total_abu[:, np.newaxis]
        
        load_ratio = total_abu.mean()/inflow.mean()

        mult_data = np.array([np.random.multinomial(10000, sample) for sample in rel_abu])
        rel_mult_data = mult_data/np.sum(mult_data, axis=1)[:, np.newaxis]

        mult_survival_idxs = np.mean(rel_mult_data, axis=0) >= 1e-3

        preference_list = np.array(dataset['CRM'])[mult_survival_idxs, :]
        cosine_similarity = sp.spatial.distance.pdist(preference_list, 'cosine')
        preference_list[preference_list > 0] = 1

        data = rel_mult_data[:, mult_survival_idxs]
        data /= np.sum(data, axis=1)[:, np.newaxis]
        
        crm = np.array(dataset['CRM'])[mult_survival_idxs, :]
        non_zero_idxs = np.sum(crm, axis=0) > 0
        
        num_resources = non_zero_idxs.size
        
        crm = crm[:, non_zero_idxs]
        crm[crm>0] = 1
        
        crm_sparsity = np.mean(crm)
        mean_num_resources = np.mean(np.sum(crm, axis=1))
        
        taylor = calc_taylor_slope(data)
        alpha = calc_alpha(data)
        env_dim, env_dim_log, env_pr, env_pr_log, env_pr_corrected, env_pr_log_correced = calc_env_dim(inflow)
        
        generalists = calc_generalists(crm, data.mean(axis=0))

        output = {
            'label': project_id,
            'num_niches': num_niches,
            'mean_load' : total_abu.mean(),
            'var_load' : np.var(total_abu),
            'load_ratio': load_ratio,
            'mean_resources' : preference_list.sum(axis=1).mean(),
            'weighted_pref' : np.dot(data.mean(axis=0), preference_list.sum(axis=1)),
            'cosine_similarity': cosine_similarity.mean(),
            'taylor': taylor,
            'alpha': alpha,
            'EnvDim': env_dim,
            'EnvDimLog': env_dim_log,
            'EnvPr': env_pr,
            'EnvPrLog': env_pr_log, 
            'EnvPrCorr': env_pr_corrected,
            'EnvPrLogCorr': env_pr_log_correced,
            'best_dist': dataset['best_dist'],
            'num_resources': num_resources,
            'generalists': generalists,
            'crm_sparsity': crm_sparsity,
            'mean_num_resources': mean_num_resources,
        }
        
        in_silico_df = pd.concat([in_silico_df, pd.DataFrame([output])], ignore_index=True)
        in_silico_df.to_csv(in_silico_checkpoint_file)

Num niches: 1


100%|██████████| 50/50 [00:01<00:00, 29.39it/s]


Num niches: 10
Already processed 0


100%|██████████| 50/50 [00:01<00:00, 25.45it/s]


Num niches: 2
Already processed 0


...

100%|██████████| 50/50 [00:01<00:00, 25.87it/s]


Num niches: 8
Already processed 0


100%|██████████| 50/50 [00:02<00:00, 19.95it/s]


Num niches: 9
Already processed 0


100%|██████████| 50/50 [00:01<00:00, 32.02it/s]


In [4]:
df_list = []

for i in tqdm(range(1, 11)):
    # Read the JSON files
    df_cg_i = pd.read_json(f'Data/in_silico_v7_results/in_silico_cg/cg_{i}.json', lines=True)
    df_cg_null_i = pd.read_json(f'Data/in_silico_v7_results/in_silico_cg_null/cg_null_{i}.json', lines=True)
    df_comp_i = pd.read_json(f'Data/in_silico_v7_results/in_silico_comp/comp_{i}.json', lines=True)
    # df_bin_pred_i = pd.read_json(f'Data/in_silico_v4_results/in_silico_bin_pred/in_silico_bin_pred_{i}.json', lines=True)
    # df_cv = pd.read_json(f'Data/in_silico_v4_results/in_silico_cv_theta/cv_theta_{i}.json', lines=True)
    # df_cv = df_cv.drop(columns=['NumNiches'])
    df_final_i = pd.merge(df_comp_i, df_cg_i, on='label')
    df_final_i = pd.merge(df_final_i, df_cg_null_i, on='label')
    # df_final_i = pd.merge(df_final_i, df_bin_pred_i, on='label')
    # df_final_i = pd.merge(df_final_i, df_cv, on='label')
    
    df_final_i['num_niches'] = i
    
    df_list.append(df_final_i)

100%|██████████| 10/10 [00:00<00:00, 46.75it/s]


In [5]:
df = pd.concat(df_list)
df['NicheDim'] = 1/df['exp_cg']
df['NicheDim_null'] = 1/df['exp_cg_null']
df['comp'] = df['comp'].apply(lambda x: np.sum(x))
df.head()

,label,comp,exp_cg,exp_cg_null,num_niches,NicheDim,NicheDim_null
0,PRJNA430990,5.226969,0.156737,0.165608,1,6.380111,6.038344
1,PRJEB27564,4.767073,0.069104,0.065894,1,14.471010,15.175921
2,PRJEB36316,5.101199,0.052274,0.055805,1,19.130015,17.919458
3,PRJNA625750,5.230733,0.117684,0.117332,1,8.497355,8.522854
4,PRJNA530790,4.551835,0.065079,0.068943,1,15.365968,14.504677


In [6]:
x = 1/df.dropna()['exp_cg']
y = df.dropna()['comp']

spearman, pval = sp.stats.spearmanr(x, y)
print(spearman, pval)

-0.3503214332857331 6.973133212696987e-16


In [6]:
in_silico_df = pd.merge(in_silico_df, df, on=['num_niches', 'label'], how='inner')
print(len(in_silico_df))

303


In [7]:
in_silico_df.to_csv(in_silico_checkpoint_file)

## Load from checkpoint

In [ ]:
in_silico_checkpoint_file = '/home/ks2823/Microbiomap/checkpoints/in_silico_v7.csv'
in_silico_df = pd.read_csv(in_silico_checkpoint_file, index_col=0)
print(in_silico_df.head())
print(len(in_silico_df))

         label  num_niches  mean_load  var_load  load_ratio  mean_resources  \
0   PRJEB27564           1   0.168827  0.000224    0.065755       30.865169   
1   PRJEB36316           1   0.258569  0.000312    0.094379       31.134831   
2  PRJNA625750           1   0.053563  0.000129    0.020171       30.783784   
3  PRJNA530790           1   0.138827  0.000105    0.055727       31.281690   
4  PRJNA674420           1   0.063023  0.000042    0.023538       31.222222   

   weighted_pref  cosine_similarity    taylor     alpha  ...      EnvPr  \
0      33.034376           0.904395  0.984401  3.056031  ...  23.132705   
1      33.732530           0.904124  0.954853  3.315368  ...  23.253927   
2      31.782954           0.904449  1.074792  2.363747  ...  24.419539   
3      33.056961           0.901581  1.002045  3.034429  ...  22.454519   
4      33.301259           0.901601  1.039687  2.882259  ...  23.295154   

    EnvPrLog  EnvPrCorr  EnvPrLogCorr  best_dist  num_resources  generalis